In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_dataset = pd.read_csv('/kaggle/input/titanic/train.csv')
test_dataset = pd.read_csv('/kaggle/input/titanic/test.csv')

train_dataset.drop(columns='Cabin',inplace=True)
train_dataset.drop(columns='Embarked',inplace=True)
test_dataset.drop(columns='Embarked',inplace=True)
test_dataset.drop(columns='Cabin',inplace=True)
#train_dataset.isna().sum()
#test_dataset.isna().sum()
train_dataset.fillna(27.0,inplace=True)
test_dataset.fillna(27.0,inplace=True)

In [3]:
y_train = train_dataset.pop('Survived')

temp=[]
i=0
while i<418:
    temp.append(1)
    i=i+1
y_test = pd.DataFrame(temp, columns=['Survived'])

In [4]:
#train_dataset.Fare.hist(bins=20)
#train_dataset.Sex.value_counts().plot(kind='barh')
#train_dataset['Pclass'].value_counts().plot(kind='barh')
#pd.concat([train_dataset, y_train], axis=1).groupby('Sex').Survived.mean().plot(kind='barh').set_xlabel('% survive')

In [5]:
CATEGORICAL_COLUMNS = ['Pclass','Sex']
NUMERIC_COLUMNS = ['Age','SibSp','Parch']

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = train_dataset[feature_name].unique() 
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

print(feature_columns)

[VocabularyListCategoricalColumn(key='Pclass', vocabulary_list=(3, 1, 2), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='Sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0), NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SibSp', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Parch', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [6]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():  
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df)) 
    if shuffle:
      ds = ds.shuffle(1000) 
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds  
  return input_function 

train_input_fn = make_input_fn(train_dataset, y_train)  
eval_input_fn = make_input_fn(test_dataset, y_test, num_epochs=1, shuffle=False)

In [7]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

In [8]:
linear_est.train(train_input_fn)

/opt/conda/lib/python3.7/site-packages/keras/engine/base_layer_v1.py:1684: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
2022-07-20 09:14:08.384518: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [9]:
pred_dicts = list(linear_est.predict(eval_input_fn))
probs = pd.Series([pred['probabilities'][1] for pred in pred_dicts])

output = test_dataset[['PassengerId']].copy()
output['Predictions'] = probs
output.to_csv("Submission.csv")

/opt/conda/lib/python3.7/site-packages/keras/engine/base_layer_v1.py:1684: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
